In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset


c:\Users\sydwi\miniconda3\envs\Env1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
df = pd.read_csv("https://huggingface.co/datasets/Looong/DMT_Assignment_2/resolve/main/submission_sample.csv")
df = pd.DataFrame(df)
print(df.head(10))

   srch_id  prop_id
0        1     3180
1        1     5543
2        1    14142
3        1    22393
4        1    24194
5        1    28181
6        1    34263
7        1    37567
8        1    50162
9        1    54937


In [17]:
print(df)

          version https://git-lfs.github.com/spec/v1
0  oid sha256:ff95de5e78d8febfce8b3a33e7a2cdd144f...
1                                      size 63842073
